# LSTM　Matchining(LSTM匹配)

In [1]:
# import modules
import pandas as pd
import numpy as np
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import nltk
import torch
import string

## 读取数据

In [2]:
datasets = pd.read_csv('datasets/DBLP-Scholar/candidates_sample.csv')
len(datasets)

1000

### 查看数据格式

In [3]:
datasets.head(10)

,Unnamed: 0,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year,label
0,5,5,conf/sigmod/2000,CpwYam_LYyEJ,proceedings of the 2000 acm sigmod internation...,NaN,2000,on predicting data cache behavior for real {ti...,"c ferdinand, r wilhelm",0,0
1,6,6,conf/sigmod/2000,F9KzvnDpCPUJ,proceedings of the 2000 acm sigmod internation...,NaN,2000,similarity-based queries for time series data....,"d raflei, a mendelzon",1997,0
2,16,16,conf/sigmod/2000,url:http://portal.acm.org/citation.cfm%3Fid%3D...,proceedings of the 2000 acm sigmod internation...,NaN,2000,source international conference on management ...,"rt snodgrass, m winslett",1994,0
3,35,35,conf/sigmod/2002,QnMrKru1S1MJ,proceedings of the 2002 acm sigmod internation...,NaN,2002,"blocation-based spatial queries,^ in proceedin...","j zhang, m zhu, d papadias, y tao, dl lee",2003,0
4,40,40,conf/sigmod/2002,url:http://portal.acm.org/citation.cfm%3Fcoll%...,proceedings of the 2002 acm sigmod internation...,NaN,2002,full text pdf format pdf (424 kb)source intern...,"c olston, a woodruff, a aiken, m chu, v ercego...",1998,0
5,42,42,conf/sigmod/2002,url:http://portal.acm.org/citation.cfm%3Fid%3D...,proceedings of the 2002 acm sigmod internation...,NaN,2002,source international conference on management ...,c zaniolo,1986,0
6,51,51,conf/sigmod/2002,url:http://portal.acm.org/citation.cfm%3Fid%3D...,proceedings of the 2002 acm sigmod internation...,NaN,2002,full text pdf format pdf (1.24 mb)source inter...,"h garcia-molina, k salem",1987,0
7,55,55,conf/sigmod/2002,vfTH6FzOpDAJ,proceedings of the 2002 acm sigmod internation...,NaN,2002,data modeling of time-based media. in: proceed...,"s gibbs, c breiteneder, d tsichritzis",1994,0
8,81,81,conf/sigmod/2003,url:http://portal.acm.org/citation.cfm%3Fid%3D...,proceedings of the 2003 acm sigmod internation...,NaN,2003,source international conference on management ...,"h boral, pa larson",1988,0
9,93,93,conf/sigmod/AbadiCCCCEGHMRSSTXYZ03,eBnT7lhV2LwJ,aurora: a data stream management system,"d abadi, d carney, u ï¿½etintemel, m cherniack...",2003,aurora: a data stream management system (demo ...,"d abadi, d carney, u cetintemel, m cherniack, c",0,1


In [4]:
data_arr = np.array(datasets)

In [5]:
data_arr.shape

(1000, 11)

In [6]:
X = data_arr[:,4:10]
y = data_arr[:,10]

In [7]:
X[19]

array(['a multimedia presentation algebra',
       's adali, m sapino, v subrahmanian', 1999,
       'a multimedia presentation algebra',
       's adalä±, ml sapino, vs subrahmanian', 1999], dtype=object)

In [8]:
y[0:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1],
      dtype=object)

In [9]:
text = nltk.word_tokenize(str(X[0][0]))
words = np.array([np.array(nltk.word_tokenize(str(x))) for x in X])

In [10]:
words[0]

array(['[', "'proceedings", 'of', 'the', '2000', 'acm', 'sigmod',
       'international', 'conference', 'on', 'management', 'of', 'data',
       ',', 'may', '16-18', ',', '2000', ',', 'dallas', ',', 'texas', ',',
       "usa'", 'nan', '2000', "'on", 'predicting', 'data', 'cache',
       'behavior', 'for', 'real', '{', 'time', 'systems', '.', 'in',
       'proceedings', 'of', 'the', 'acm', 'sigplan', 'workshop', "'", "'",
       'c', 'ferdinand', ',', 'r', 'wilhelm', "'", '0', ']'], dtype='<U13')

## 测试bert

### 加载预处理的Bert模型

以下是如何获取支持pytorch支持的Bert模型

1. [下载Bert模型](https://github.com/google-research/bert#pre-trained-models)

2. [转换Bert模型支持pytorch](https://blog.csdn.net/weixin_41287060/article/details/105080705)

In [15]:
# 使用下载的本地模型,加快速度
tokenizer = BertTokenizer.from_pretrained('cased_L-12_H-768_A-12')
model = BertModel.from_pretrained('cased_L-12_H-768_A-12')

### 获取隐藏层向量

In [16]:
text = str(words[0])
tokenized_text = tokenizer.tokenize(text) #token初始化
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) #获取词汇表索引
tokens_tensor = torch.tensor([indexed_tokens]) #将输入转化为torch的tensor
with torch.no_grad(): #禁用梯度计算 因为只是前向传播获取隐藏层状态，所以不需要计算梯度
    last_hidden_states = model(tokens_tensor)[0]

In [17]:
last_hidden_states[0].shape

torch.Size([1, 191, 768])

In [18]:
data_arr[4][2]

'conf/sigmod/2002'

In [19]:
import nltk
import re

In [20]:
data_arr[4][4]

'proceedings of the 2002 acm sigmod international conference on management of data, madison, wisconsin, june 3-6, 2002'

In [21]:
nltk.word_tokenize('Life is like a box of chocolate, you never know what you gonna get.')

['Life',
 'is',
 'like',
 'a',
 'box',
 'of',
 'chocolate',
 ',',
 'you',
 'never',
 'know',
 'what',
 'you',
 'gon',
 'na',
 'get',
 '.']